In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter

In [2]:
pl_index = 2340
song_num = 20
tag_num = 10

In [3]:
f = pd.read_json('../kakao_arena_melon/arena_data/orig/train.json' ,typ = 'frame', encoding='utf-8')
df = pd.DataFrame(f)
df = df.sort_values(by=['like_cnt'],ascending=False)
df = df[df['like_cnt']>5]

# 플레이리스트 내 song id 리스트
song_all = df['songs']
# 플레이리스트 내 tag 리스트
tag_all = df['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in song_all for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in tag_all for tag in plist]

# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

# 전체 플레이리스트 내 20회 넘게 담긴 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    if value>20:
        mfl[key]=value
song_len = len(mfl) # 20회 넘게 담긴 song의 개수
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
    if value>10:
        mfl[key]=value
tag_len=len(mfl)-song_len # 10회 넘게 담긴 tag의 개수
mfl_col = list(mfl.keys()) # 유의미한 song과 tag 전체 개수

# 각 플레이리스트에 해당하는 곡, 태그 좌표에 1 부여
def create_onehot(column_name, pl_index):
    zero_matrix=np.zeros((1,len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name)
    for i in df.iloc[pl_index,0]+df.iloc[pl_index,3] :
        if i in column_name:
            zero_df.iloc[0,column_name.index(i)]=1
    return zero_df

autoencoder = tf.keras.models.load_model('autoencoder_basic.h5')
predict_plist = autoencoder.predict(create_onehot(mfl_col, pl_index))

ori_song = mfl_col[:song_len]
ori_tag = mfl_col[song_len:]
song_predict = predict_plist[:,:song_len] # song output(추천곡)
tag_predict = predict_plist[:,song_len:] # tag output(추천태그)

plist_song=song_predict[0].argsort()[-song_num:] # predict한 song output 중 상위 n개
p_song=[]
for song in plist_song:
    p_song.append(ori_song[song])
    
plist_tag=tag_predict[0].argsort()[-tag_num:] # predict한 tag output 중 상위 n개
p_tag=[]
for tag in plist_tag:
    p_tag.append(ori_tag[tag])

f_meta = pd.read_json('../kakao_arena_melon/res/song_meta.json' ,typ = 'frame', encoding='utf-8')
df_meta = pd.DataFrame(f_meta)

1/1 [==============================] - 0s 199ms/step


In [4]:
# Input Playlist - song
df_meta[df_meta['id'].isin(df.iloc[pl_index,:]['songs'])][['artist_name_basket', 'song_name']]

,artist_name_basket,song_name
1508,[롤러 코스터],숨길 수 없어요
11761,[권순관],Tonight
32557,[존박],그 노래
33829,[스웨덴세탁소],목소리 (Feat. 정기고)
34340,[심규선 (Lucia)],녹여줘
...,...,...
652479,[그_냥],부드러운
681080,[참깨와 솜사탕],공놀이
682900,[크래커 (CRACKER)],아 (Feat. Jungmin)
691256,[S.M. THE BALLAD],Set Me Free (Sung By 태연 (소녀시대))


In [5]:
# Input Playlist - tag
df.iloc[pl_index,:]['tags']

['잔잔한', '밤', '새벽']

In [6]:
# Output Playlist - song
df_meta[df_meta['id'].isin(p_song)][['artist_name_basket', 'song_name']]

,artist_name_basket,song_name
27469,[백예린],Bye bye my blue
33829,[스웨덴세탁소],목소리 (Feat. 정기고)
53849,[offonoff (오프온오프)],bath
75842,[정키],잊혀지다 (Feat. 양다일)
90557,[어쿠루브],하고 싶은 말 (Feat. 한올)
116573,[정준일],안아줘
141248,[Crush],SOFA
290353,[정키],I`m Sorry (Feat. 웰던포테이토)
339124,"[Zion.T, Crush]",그냥 (Just)
366786,[가을방학],가끔 미치도록 네가 안고 싶어질 때가 있어


In [7]:
# Output Playlist - tag
p_tag

['우울', '알앤비', '비오는날', '새벽감성', '감성', '슬픔', '이별', '잔잔한', '밤', '새벽']